In [99]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

In [100]:
data = pd.read_csv("sph6004_assignment1_data.csv")
df = pd.DataFrame(data)
df

,id,hospital_mortality,aki_stage,gender,admission_age,race,heart_rate_min,heart_rate_max,heart_rate_mean,sbp_min,...,ggt_max,ld_ldh_min,ld_ldh_max,gcs_min,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,height,weight_admit
0,39553978,0,0,F,52.559969,WHITE,91.0,105.0,96.500000,82.0,...,NaN,173.0,173.0,14.0,6.0,4.0,4.0,0.0,152.0,39.4
1,39765666,0,0,F,76.486231,BLACK/AFRICAN AMERICAN,68.0,80.0,73.636364,130.0,...,NaN,NaN,NaN,15.0,6.0,5.0,4.0,0.0,NaN,76.2
2,37067082,0,0,F,55.881486,WHITE,78.0,106.0,93.200000,117.0,...,NaN,NaN,NaN,15.0,6.0,5.0,4.0,0.0,NaN,71.2
3,31205490,0,0,F,46.275517,WHITE,55.0,91.0,79.480000,73.0,...,NaN,NaN,NaN,15.0,6.0,5.0,4.0,0.0,157.0,72.2
4,37510196,1,2,F,77.018296,BLACK/AFRICAN AMERICAN,38.0,80.0,71.392857,93.0,...,NaN,NaN,NaN,15.0,1.0,0.0,1.0,1.0,157.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50915,32336619,0,0,M,43.884161,WHITE,76.0,110.0,87.758621,106.0,...,NaN,NaN,NaN,10.0,5.0,2.0,3.0,0.0,193.0,107.5
50916,31070865,0,0,M,82.773493,WHITE,63.0,96.0,71.000000,91.0,...,NaN,NaN,NaN,11.0,5.0,2.0,4.0,0.0,NaN,50.5
50917,36075953,0,0,F,48.022487,WHITE,93.0,109.0,100.423077,91.0,...,NaN,238.0,238.0,14.0,6.0,5.0,3.0,0.0,NaN,67.9
50918,38978960,1,2,M,58.694264,WHITE,71.0,100.0,84.666667,75.0,...,NaN,NaN,NaN,5.0,3.0,1.0,1.0,0.0,178.0,77.5


In [101]:
df.shape
df.isnull().sum()

id                        0
hospital_mortality        0
aki_stage                 0
gender                    0
admission_age             0
                      ...  
gcs_verbal              513
gcs_eyes                378
gcs_unable              235
height                25909
weight_admit           1020
Length: 165, dtype: int64

In [102]:
def race(r):
    if r != 'WHITE' and r != 'BLACK/AFRICAN AMERICAN':
        return 'others'
    else:
        return r
df['race'] = df['race'].apply(race)

encoder = OneHotEncoder(drop="first", sparse_output=False)
race_encoded = encoder.fit_transform(df[['race']])
race_encoded_df = pd.DataFrame(race_encoded, columns=encoder.get_feature_names_out(['race']))

df = df.drop(columns=['race']).reset_index(drop=True)
df = pd.concat([df, race_encoded_df], axis=1)

df['gender'] = df['gender'].replace({'F':0,'M':1})

df.head()

C:\Users\matebook 14\AppData\Local\Temp\ipykernel_28132\3305361035.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['gender'] = df['gender'].replace({'F':0,'M':1})


,id,hospital_mortality,aki_stage,gender,admission_age,heart_rate_min,heart_rate_max,heart_rate_mean,sbp_min,sbp_max,...,ld_ldh_max,gcs_min,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,height,weight_admit,race_WHITE,race_others
0,39553978,0,0,0,52.559969,91.0,105.0,96.500000,82.0,95.0,...,173.0,14.0,6.0,4.0,4.0,0.0,152.0,39.4,1.0,0.0
1,39765666,0,0,0,76.486231,68.0,80.0,73.636364,130.0,158.0,...,NaN,15.0,6.0,5.0,4.0,0.0,NaN,76.2,0.0,0.0
2,37067082,0,0,0,55.881486,78.0,106.0,93.200000,117.0,151.0,...,NaN,15.0,6.0,5.0,4.0,0.0,NaN,71.2,1.0,0.0
3,31205490,0,0,0,46.275517,55.0,91.0,79.480000,73.0,115.0,...,NaN,15.0,6.0,5.0,4.0,0.0,157.0,72.2,1.0,0.0
4,37510196,1,2,0,77.018296,38.0,80.0,71.392857,93.0,180.0,...,NaN,15.0,1.0,0.0,1.0,1.0,157.0,65.0,0.0,0.0


In [103]:
X = df.drop(columns='aki_stage')
y = df['aki_stage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

threshold = 0.5
missing_ratio = X_train.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio >= threshold].index
X_train.drop(columns=cols_to_drop, inplace=True)

X_test.drop(columns=cols_to_drop, inplace=True)

X_train.drop(columns='id', inplace=True)
X_test.drop(columns='id', inplace=True)

var_thresh = VarianceThreshold(threshold=0.5)
var_thresh.fit(X_train)

selected_features = X_train.columns[var_thresh.get_support()]

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

X_train_selected.head()

,admission_age,heart_rate_min,heart_rate_max,heart_rate_mean,sbp_min,sbp_max,sbp_mean,dbp_min,dbp_max,dbp_mean,...,inr_max,pt_min,pt_max,ptt_min,ptt_max,gcs_min,gcs_motor,gcs_verbal,gcs_eyes,weight_admit
21594,80.746049,78.0,119.0,99.666667,45.000000,137.0,104.527027,34.0,66.0,52.770270,...,1.4,12.6,15.3,23.2,150.0,15.0,6.0,0.0,3.0,67.1
47259,42.783657,98.0,139.0,112.500000,81.000000,120.0,98.086957,57.0,105.0,68.434783,...,1.7,17.3,18.4,29.2,52.6,13.0,6.0,4.0,3.0,61.7
17164,59.614632,82.0,114.0,95.945946,84.333333,144.5,120.565892,51.0,86.0,61.562016,...,1.3,14.2,15.1,35.2,38.8,10.0,6.0,1.0,3.0,84.0
27813,75.150485,71.0,90.0,80.580645,82.000000,149.0,126.222222,50.0,100.0,68.277778,...,3.3,14.1,33.6,28.4,40.4,14.0,6.0,4.0,4.0,68.0
10552,63.367793,61.0,92.0,75.880000,83.000000,176.0,126.250000,68.0,99.0,83.166667,...,NaN,NaN,NaN,NaN,NaN,15.0,1.0,0.0,1.0,94.0


In [104]:
for category in y_train.unique():

    X_train_group = X_train_selected[y_train == category]

    medians = X_train_group.median()

    X_train_selected.loc[y_train == category] = X_train_selected.loc[y_train == category].fillna(medians)

for category in y_train.unique():

    X_test_group = X_test_selected[y_test == category]

    medians = X_train_selected.loc[y_train == category].median()

    X_test_selected.loc[y_test == category] = X_test_selected.loc[y_test == category].fillna(medians)

X_train_selected.head()

,admission_age,heart_rate_min,heart_rate_max,heart_rate_mean,sbp_min,sbp_max,sbp_mean,dbp_min,dbp_max,dbp_mean,...,inr_max,pt_min,pt_max,ptt_min,ptt_max,gcs_min,gcs_motor,gcs_verbal,gcs_eyes,weight_admit
21594,80.746049,78.0,119.0,99.666667,45.000000,137.0,104.527027,34.0,66.0,52.770270,...,1.4,12.6,15.3,23.2,150.0,15.0,6.0,0.0,3.0,67.1
47259,42.783657,98.0,139.0,112.500000,81.000000,120.0,98.086957,57.0,105.0,68.434783,...,1.7,17.3,18.4,29.2,52.6,13.0,6.0,4.0,3.0,61.7
17164,59.614632,82.0,114.0,95.945946,84.333333,144.5,120.565892,51.0,86.0,61.562016,...,1.3,14.2,15.1,35.2,38.8,10.0,6.0,1.0,3.0,84.0
27813,75.150485,71.0,90.0,80.580645,82.000000,149.0,126.222222,50.0,100.0,68.277778,...,3.3,14.1,33.6,28.4,40.4,14.0,6.0,4.0,4.0,68.0
10552,63.367793,61.0,92.0,75.880000,83.000000,176.0,126.250000,68.0,99.0,83.166667,...,1.3,13.1,14.3,28.4,31.7,15.0,1.0,0.0,1.0,94.0


In [105]:
corr_matrix = X_train_selected.corr()

upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

threshold = 0.6

to_drop = [column for column in upper_tri.columns if any(upper_tri[column].abs() > threshold)]

X_train_selected = X_train_selected.drop(columns=to_drop)

X_test_selected = X_test_selected.drop(columns=to_drop, errors='ignore')

X_test_selected.head()

,admission_age,heart_rate_min,heart_rate_max,sbp_min,sbp_max,dbp_min,dbp_max,resp_rate_min,resp_rate_max,temperature_vital_min,...,potassium_lab_max,abs_lymphocytes_min,abs_monocytes_min,abs_neutrophils_min,inr_max,ptt_min,ptt_max,gcs_min,gcs_motor,weight_admit
47134,58.425782,78.0,142.0,111.0,185.0,56.0,95.0,12.0,26.0,35.39,...,4.4,0.6386,0.1860,5.3320,1.3,28.4,31.7,15.0,6.0,114.0
7547,64.964858,58.0,78.0,65.0,138.0,45.0,89.0,6.0,27.0,36.28,...,5.6,1.1768,0.4800,8.7344,1.2,37.5,38.2,15.0,1.0,75.8
20518,83.270679,84.0,110.0,98.0,123.0,48.0,81.0,16.0,29.0,36.06,...,4.2,1.2852,0.4547,7.6400,1.1,30.8,70.7,15.0,6.0,34.5
37776,54.820712,53.0,78.0,90.0,126.0,47.0,74.0,17.0,28.0,35.56,...,3.7,0.4980,0.1050,2.3610,1.3,31.2,31.2,15.0,6.0,71.2
27553,61.311234,61.0,90.0,74.0,127.0,47.0,68.0,12.0,23.0,35.60,...,4.1,2.1500,1.5100,17.1600,1.6,28.0,31.8,15.0,6.0,106.6


In [106]:
class GreedyFeatureSelection:
    def evaluate_score(self, X_train, y_train, X_val, y_val):

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        model = LogisticRegression()
        model.fit(X_train_scaled, y_train)

        predictions = model.predict_proba(X_val_scaled)[:, 1]
        auc = metrics.roc_auc_score(y_val, predictions)
        return auc

    def _feature_selection(self, X_train, y_train, delta_threshold=0.001):
        good_features = []
        best_scores = []
        num_features = X_train.shape[1]

        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        while True:
            this_feature = None
            best_score = 0
            for feature in range(num_features):
                if feature in good_features:
                    continue
                selected_features = good_features + [feature]
                xtrain = X_train.iloc[:, selected_features]

                fold_scores = []
                for train_idx, val_idx in kf.split(xtrain, y_train):
                    X_train_fold, X_val_fold = xtrain.iloc[train_idx], xtrain.iloc[val_idx]
                    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

                    score = self.evaluate_score(X_train_fold, y_train_fold, X_val_fold, y_val_fold)
                    fold_scores.append(score)

                avg_score = np.mean(fold_scores)

                if avg_score > best_score:
                    this_feature = feature
                    best_score = avg_score

            if this_feature is None:
                break

            good_features.append(this_feature)
            best_scores.append(best_score)

            if len(best_scores) > 1 and (best_scores[-1] - best_scores[-2]) < delta_threshold:
                break

        return best_scores[:-1], good_features[:-1]

    def __call__(self, X_train, y_train, delta_threshold=0.001):
        scores, feature_indices = self._feature_selection(X_train, y_train, delta_threshold)
        selected_features = X_train.columns[feature_indices]
        return X_train.iloc[:, feature_indices], selected_features, scores


def map_aki(aki):
    return 1 if aki != 0 else 0

y_logis = y_train.apply(map_aki)

selector = GreedyFeatureSelection()
X_train_transformed, selected_features, scores = selector(X_train_selected, y_logis, delta_threshold=0.001)

print("Selected Features:", selected_features.tolist())
print("AUC Score:", scores)
print("Feature Number:", len(selected_features.tolist()))

Selected Features: ['pco2_max', 'gcs_motor', 'bun_min', 'weight_admit', 'sbp_min', 'admission_age', 'temperature_vital_max', 'pco2_min', 'sbp_max', 'ptt_max', 'abs_neutrophils_min', 'potassium_lab_max', 'spo2_max', 'heart_rate_max', 'inr_max', 'chloride_lab_min', 'po2_max', 'po2_min']
AUC Score: [0.6462031200013977, 0.6663015476820406, 0.6710428971163148, 0.6864013376622672, 0.7028525910709762, 0.7166115136916262, 0.7238322335387577, 0.7281553905684272, 0.7320143866589918, 0.7351271661248496, 0.7381501325115292, 0.7403523670515203, 0.7421340398178689, 0.7439889310092995, 0.7453758223180678, 0.7464590011966391, 0.7476387279820662, 0.7489616030569]
Feature Number: 18


In [107]:
X_transform_train = X_train_selected[selected_features]
X_transform_test = X_test_selected[selected_features]
X_transform_train.head()

,pco2_max,gcs_motor,bun_min,weight_admit,sbp_min,admission_age,temperature_vital_max,pco2_min,sbp_max,ptt_max,abs_neutrophils_min,potassium_lab_max,spo2_max,heart_rate_max,inr_max,chloride_lab_min,po2_max,po2_min
21594,54.0,6.0,13.0,67.1,45.000000,80.746049,36.89,42.0,137.0,150.0,12.5135,7.6,100.0,119.0,1.4,89.0,252.0,31.0
47259,34.0,6.0,7.0,61.7,81.000000,42.783657,38.56,34.0,120.0,52.6,5.8000,4.3,98.0,139.0,1.7,98.0,43.0,43.0
17164,50.0,6.0,16.0,84.0,84.333333,59.614632,37.70,28.0,144.5,38.8,7.6400,4.3,100.0,114.0,1.3,107.0,410.0,120.0
27813,46.0,6.0,11.0,68.0,82.000000,75.150485,37.00,36.0,149.0,40.4,9.0180,4.6,100.0,90.0,3.3,104.0,208.0,79.0
10552,52.0,1.0,16.0,94.0,83.000000,63.367793,37.28,35.0,176.0,31.7,10.0457,8.0,100.0,92.0,1.3,103.0,163.0,89.0


In [95]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf_model = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')

grid_search = RandomizedSearchCV(rf_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_transform_train, y_train)

print("Optimal Params:", grid_search.best_params_)
best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_transform_test)

accuracy = accuracy_score(y_test, y_pred)
print("Optimal Accuracy:", accuracy)

print("Classification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Optimal Params: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30}
Optimal Accuracy: 0.7935978004713276
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.86      0.86      3310
           1       0.89      0.66      0.75      1972
           2       0.74      0.83      0.78      3327
           3       0.71      0.75      0.73      1575

    accuracy                           0.79     10184
   macro avg       0.80      0.77      0.78     10184
weighted avg       0.80      0.79      0.79     10184



In [96]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

grid_search = RandomizedSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_transform_train, y_train)

print("Optimal Params:", grid_search.best_params_)
best_xgb = grid_search.best_estimator_

y_pred = best_xgb.predict(X_transform_test)

accuracy = accuracy_score(y_test, y_pred)
print("Optimal Accuracy:", accuracy)

print("Classification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


D:\workshop_3\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [14:20:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Optimal Params: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': None}
Optimal Accuracy: 0.799096622152396
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.86      0.86      3310
           1       0.85      0.69      0.76      1972
           2       0.75      0.83      0.79      3327
           3       0.73      0.73      0.73      1575

    accuracy                           0.80     10184
   macro avg       0.80      0.78      0.79     10184
weighted avg       0.80      0.80      0.80     10184



In [108]:
from lightgbm import LGBMClassifier
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, -1],
    'num_leaves': [31, 50, 70, 100],
    'min_child_samples': [10, 20, 30],
    'feature_fraction': [0.8, 0.9, 1.0]
}

lgbm_model = LGBMClassifier(random_state=42, is_unbalance=True,class_weight='balanced')

grid_search = RandomizedSearchCV(lgbm_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_transform_train, y_train)

print("Optimal Params:", grid_search.best_params_)
best_lgbm = grid_search.best_estimator_

y_pred = best_lgbm.predict(X_transform_test)

accuracy = accuracy_score(y_test, y_pred)
print("Optimal Accuracy:", accuracy)

print("Classification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003734 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2927
[LightGBM] [Info] Number of data points in the train set: 40736, number of used features: 18
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
Optimal Params: {'num_leaves': 100, 'n_estimators': 300, 'min_child_samples': 20, 'max_depth': 20, 'feature_fraction': 1.0